In [2]:
import pandas as pd
import numpy as np
import random
import uuid  # Ajout de l'import uuid pour générer des adresses email uniques

# --- Paramètres Généraux ---
N_RECORDS = 40000
FILENAME = "dataset_revenu_marocains.csv"
RANDOM_SEED = 42 # Pour la reproductibilité
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# --- Définition des Catégories ---
MILIEU_OPTS = ['Urbain', 'Rural']
SEXE_OPTS = ['Homme', 'Femme']
NIVEAU_EDUCATION_OPTS = ['Sans niveau', 'Fondamental', 'Secondaire', 'Supérieur']
ETAT_MATRIMONIAL_OPTS = ['Célibataire', 'Marié', 'Divorcé', 'Veuf']
CSP_OPTS = ['Cadres supérieurs', 'Professions intermédiaires', 'Employés', 'Ouvriers', 'Agriculteurs', 'Inactifs']
REGION_GEO_OPTS = ['Nord', 'Centre', 'Sud', 'Est', 'Ouest']
SECTEUR_EMPLOI_OPTS = ['Public', 'Privé', 'Informel']
OUI_NON_OPTS = ['Oui', 'Non']

# --- Proportions pour la génération (peuvent être ajustées) ---
P_URBAIN = 0.60 # Proportion cible d'urbains (ajustable)
P_RURAL = 1 - P_URBAIN

# --- Fonctions de Génération des Caractéristiques (similaires à v1, avec ajustements si besoin) ---

def generate_age(n):
    return np.random.randint(18, 64, n)

def generate_milieu(n):
    return np.random.choice(MILIEU_OPTS, n, p=[P_URBAIN, P_RURAL])

def generate_sexe(n):
    return np.random.choice(SEXE_OPTS, n, p=[0.52, 0.48])

def generate_niveau_education(n):
    return np.random.choice(NIVEAU_EDUCATION_OPTS, n, p=[0.15, 0.30, 0.35, 0.20]) # Un peu plus de 'Supérieur'

def generate_annees_experience(age, niveau_education):
    experience = []
    for a, edu in zip(age, niveau_education):
        min_age_travail = 18
        if edu == 'Supérieur': min_age_travail = 23
        elif edu == 'Secondaire': min_age_travail = 20
        
        max_exp = a - min_age_travail
        if max_exp < 0: max_exp = 0
        
        exp = np.random.randint(0, max_exp + 1) if max_exp > 0 else 0
        exp = min(exp, a - 16) if a > 16 else 0
        experience.append(max(0,exp))
    return np.array(experience)

def generate_etat_matrimonial(n, age):
    etats = []
    for a in age:
        if a < 25: etats.append(np.random.choice(ETAT_MATRIMONIAL_OPTS, p=[0.8, 0.15, 0.03, 0.02]))
        elif a < 45: etats.append(np.random.choice(ETAT_MATRIMONIAL_OPTS, p=[0.2, 0.65, 0.1, 0.05]))
        else: etats.append(np.random.choice(ETAT_MATRIMONIAL_OPTS, p=[0.1, 0.55, 0.15, 0.2]))
    return np.array(etats)

def generate_csp(niveau_education, annees_experience, age):
    csps = []
    for edu, exp, a in zip(niveau_education, annees_experience, age):
        if exp < 1 and a < 22 and edu in ['Sans niveau', 'Fondamental']: csps.append('Inactifs')
        elif edu == 'Supérieur':
            if exp > 8: csps.append('Cadres supérieurs')
            elif exp > 2: csps.append('Professions intermédiaires')
            else: csps.append(np.random.choice(['Employés', 'Professions intermédiaires'], p=[0.7,0.3]))
        elif edu == 'Secondaire':
            if exp > 12: csps.append('Professions intermédiaires')
            elif exp > 4: csps.append('Employés')
            else: csps.append(np.random.choice(['Ouvriers','Employés'], p=[0.7,0.3]))
        elif edu == 'Fondamental':
            if exp > 8: csps.append('Ouvriers')
            else: csps.append(np.random.choice(['Ouvriers', 'Agriculteurs'], p=[0.6,0.4]))
        else: # Sans niveau
            csps.append(np.random.choice(['Ouvriers', 'Agriculteurs', 'Inactifs'], p=[0.35,0.35,0.3]))
    return np.array(csps)

def generate_possession_biens(n, csp_list, milieu_list):
    prop_immo, veh_motor, terr_agri = [], [], []
    for csp, milieu in zip(csp_list, milieu_list):
        p_immo, p_veh, p_agri = 0.05, 0.05, 0.02
        if csp == 'Cadres supérieurs': p_immo, p_veh = 0.7, 0.8
        elif csp == 'Professions intermédiaires': p_immo, p_veh = 0.5, 0.6
        elif csp == 'Employés': p_immo, p_veh = 0.25, 0.35
        elif csp == 'Ouvriers': p_immo, p_veh = 0.1, 0.15
        elif csp == 'Agriculteurs': p_immo, p_veh, p_agri = 0.3, 0.25, 0.6
        
        if milieu == 'Rural':
            p_veh *= 0.7
            if csp == 'Agriculteurs': p_agri = 0.8
        
        prop_immo.append(np.random.choice(OUI_NON_OPTS, p=[p_immo, 1-p_immo]))
        veh_motor.append(np.random.choice(OUI_NON_OPTS, p=[p_veh, 1-p_veh]))
        terr_agri.append(np.random.choice(OUI_NON_OPTS, p=[p_agri, 1-p_agri]))
    return prop_immo, veh_motor, terr_agri

def generate_region_geographique(n):
    return np.random.choice(REGION_GEO_OPTS, n, p=[0.22, 0.28, 0.15, 0.15, 0.20]) # Nord, Centre, Sud, Est, Ouest

def generate_secteur_emploi(n, csp_list):
    secteurs = []
    for csp in csp_list:
        if csp == 'Inactifs': secteurs.append(np.nan)
        elif csp == 'Agriculteurs': secteurs.append(np.random.choice(['Privé', 'Informel'], p=[0.2,0.8]))
        elif csp in ['Cadres supérieurs', 'Professions intermédiaires']: secteurs.append(np.random.choice(['Public', 'Privé'], p=[0.35, 0.65]))
        elif csp == 'Employés': secteurs.append(np.random.choice(['Public', 'Privé', 'Informel'], p=[0.3, 0.5, 0.2]))
        else: secteurs.append(np.random.choice(['Privé', 'Informel'], p=[0.4, 0.6]))
    return np.array(secteurs)

def generate_revenu_secondaire(n, csp_list):
    probs = []
    for csp in csp_list:
        if csp in ['Cadres supérieurs', 'Professions intermédiaires']: probs.append(0.35)
        elif csp == 'Agriculteurs': probs.append(0.25)
        elif csp == 'Employés': probs.append(0.15)
        else: probs.append(0.05)
    return np.array([np.random.choice(OUI_NON_OPTS, p=[p, 1-p]) for p in probs])

# --- Nouvelle Fonction de Génération du Revenu Annuel ---
def generate_revenu_annuel(df):
    n = len(df)
    revenus = np.zeros(n)

    # Revenu de base avec moins de variance
    base_revenu = np.random.lognormal(mean=np.log(1500), sigma=0.05, size=n)  # Réduction de sigma

    for i in range(n):
        record = df.iloc[i]
        rev_i = base_revenu[i]

        # Impact du Milieu
        milieu_bonus = 2000 if record['Milieu'] == 'Urbain' else 500  # Valeurs fixes
        rev_i += milieu_bonus

        # Impact Niveau d'Éducation
        if record['Niveau_education'] == 'Supérieur':
            rev_i += 15000 * (1 + 0.1 * record['Annees_experience'])
        elif record['Niveau_education'] == 'Secondaire':
            rev_i += 6000 * (1 + 0.07 * record['Annees_experience'])
        elif record['Niveau_education'] == 'Fondamental':
            rev_i += 2000 * (1 + 0.05 * record['Annees_experience'])
        else:
            rev_i += 500 * (1 + 0.03 * record['Annees_experience'])

        # Impact CSP
        csp_factor = {
            'Cadres supérieurs': 2.0,
            'Professions intermédiaires': 1.6,
            'Employés': 1.2,
            'Ouvriers': 0.9,
            'Agriculteurs': 0.8,
            'Inactifs': 1.0
        }.get(record['CSP'], 1.0)
        rev_i *= csp_factor

        # Impact Sexe
        rev_i *= 1.2 if record['Sexe'] == 'Homme' else 0.95

        # Impact Région
        region_factor = {
            'Centre': 1.1,
            'Ouest': 1.1,
            'Nord': 1.05,
            'Sud': 0.95,
            'Est': 0.95
        }.get(record['Region_geographique'], 1.0)
        rev_i *= region_factor

        # Impact Secteur Emploi
        if pd.notna(record['Secteur_emploi']) and record['CSP'] != 'Inactifs':
            secteur_factor = {
                'Public': 1.05,
                'Privé': 1.1,
                'Informel': 0.8
            }.get(record['Secteur_emploi'], 1.0)
            rev_i *= secteur_factor

        # Impact Revenu Secondaire
        if record['Revenu_secondaire'] == 'Oui':
            rev_i += 3000  # Valeur fixe

        # Plafonnement et plancher
        rev_i = max(300, min(rev_i, 600000))
        revenus[i] = round(rev_i, 0)

    return revenus

# --- Fonctions pour Imperfections (améliorées) ---
def add_valeurs_manquantes(df, colonnes_pour_nan, p_nan=0.001):
    """Ajoute des valeurs manquantes (NaN) de manière aléatoire dans les colonnes spécifiées."""
    for col in colonnes_pour_nan:
        if col in df.columns:
            mask = np.random.choice([True, False], size=len(df), p=[p_nan, 1-p_nan])
            df.loc[mask, col] = np.nan
    return df

def add_valeurs_aberrantes_age(df, p_aberrant=0.001):
    """Ajoute des valeurs aberrantes dans la colonne 'Age'."""
    n_aberr = int(p_aberrant * len(df))
    valid_indices = df.index.tolist()
    aberr_indices = random.sample(valid_indices, min(n_aberr, len(valid_indices)))
    for idx in aberr_indices:
        df.loc[idx, 'Age'] = np.random.choice([-5, 150])  # Valeurs aberrantes
    return df

def add_valeurs_aberrantes_experience(df, p_aberrant=0.001):
    """Ajoute des valeurs aberrantes dans la colonne 'Annees_experience', en tenant compte de l'âge."""
    n_aberr = int(p_aberrant * len(df))
    valid_indices = df.index.tolist()
    aberr_indices = random.sample(valid_indices, min(n_aberr, len(valid_indices)))
    for idx in aberr_indices:
        age = df.loc[idx, 'Age']
        df.loc[idx, 'Annees_experience'] = age + np.random.randint(10, 30)  # Expérience > Age
    return df

def add_valeurs_aberrantes_possession(df, p_aberrant=0.001):
    """Ajoute des valeurs aberrantes dans les colonnes de possession de biens, en tenant compte de la CSP."""
    n_aberr = int(p_aberrant * len(df))
    valid_indices = df.index.tolist()
    aberr_indices = random.sample(valid_indices, min(n_aberr, len(valid_indices)))
    for idx in aberr_indices:
        csp = df.loc[idx, 'CSP']
        if csp == 'Inactifs':
            df.loc[idx, 'Propriete_immobiliere'] = np.random.choice(['Oui', 'Non'], p=[0.9, 0.1])
            df.loc[idx, 'Vehicule_motorise'] = np.random.choice(['Oui', 'Non'], p=[0.8, 0.2])
            df.loc[idx, 'Terrain_agricole'] = np.random.choice(['Oui', 'Non'], p=[0.1, 0.9])
        elif csp == 'Agriculteurs':
            df.loc[idx, 'Terrain_agricole'] = np.random.choice(['Oui', 'Non'], p=[0.05, 0.95])
    return df

def add_valeurs_aberrantes_revenu(revenus_col, csp_col, p_aberrant=0.0005):
    """Ajoute des valeurs aberrantes dans la colonne 'Revenu_Annuel', en tenant compte de la CSP."""
    n_aberr = int(p_aberrant * len(revenus_col))
    valid_indices = revenus_col[revenus_col.notna()].index.tolist()
    aberr_indices = random.sample(valid_indices, min(n_aberr, len(valid_indices)))
    
    for idx in aberr_indices:
        csp = csp_col.loc[idx]
        if csp == 'Inactifs':
            revenus_col.loc[idx] = np.random.choice([100, 200, 300])
        elif csp == 'Cadres supérieurs':
            revenus_col.loc[idx] = np.random.choice([1000000, 1200000, 1500000])
        elif csp == 'Professions intermédiaires':
            revenus_col.loc[idx] = np.random.choice([600000, 700000])
        elif csp == 'Employés':
            revenus_col.loc[idx] = np.random.choice([400000, 500000])
        elif csp == 'Ouvriers':
            revenus_col.loc[idx] = np.random.choice([200000, 300000])
        elif csp == 'Agriculteurs':
            revenus_col.loc[idx] = np.random.choice([50000, 60000])
        else:
            revenus_col.loc[idx] = np.random.choice([300000, 400000, 500000])
    return revenus_col

# --- Génération du Dataset  ---
def generate_dataset():
    data = pd.DataFrame()

    data['Age'] = generate_age(N_RECORDS)
    data['Milieu'] = generate_milieu(N_RECORDS)
    data['Sexe'] = generate_sexe(N_RECORDS)
    data['Niveau_education'] = generate_niveau_education(N_RECORDS)
    data['Annees_experience'] = generate_annees_experience(data['Age'], data['Niveau_education'])
    data['Etat_matrimonial'] = generate_etat_matrimonial(N_RECORDS, data['Age'])
    data['CSP'] = generate_csp(data['Niveau_education'], data['Annees_experience'], data['Age'])
    
    prop_immo, veh_motor, terr_agri = generate_possession_biens(N_RECORDS, data['CSP'], data['Milieu'])
    data['Propriete_immobiliere'] = prop_immo
    data['Vehicule_motorise'] = veh_motor
    data['Terrain_agricole'] = terr_agri

    data['Region_geographique'] = generate_region_geographique(N_RECORDS)
    data['Secteur_emploi'] = generate_secteur_emploi(N_RECORDS, data['CSP'])
    data['Revenu_secondaire'] = generate_revenu_secondaire(N_RECORDS, data['CSP'])
    
    data['Revenu_Annuel'] = generate_revenu_annuel(data.copy()) # Utilise la nouvelle fonction

    # Imperfections
    cols_with_nan = ['Etat_matrimonial', 'Secteur_emploi', 'Revenu_secondaire', 
                     'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole',
                     'Annees_experience'] # CSP peut avoir des Inactifs, pas besoin de NaN explicite
    data = add_valeurs_manquantes(data, cols_with_nan, p_nan=0.001)
    
    # Aberrations sur le revenu (après génération et NaN)
    revenu_not_na_mask = data['Revenu_Annuel'].notna()
    data.loc[revenu_not_na_mask, 'Revenu_Annuel'] = add_valeurs_aberrantes_revenu(
        data.loc[revenu_not_na_mask, 'Revenu_Annuel'].copy(),
        data.loc[revenu_not_na_mask, 'CSP'].copy()
    )

    # AJOUT : Aberrations sur d'autres colonnes
    data = add_valeurs_aberrantes_age(data, p_aberrant=0.001)
    data = add_valeurs_aberrantes_experience(data, p_aberrant=0.001)
    data = add_valeurs_aberrantes_possession(data, p_aberrant=0.001)

    # Colonne dérivée utile pour EDA, mais pas pour modélisation directe si Age est déjà là
    bins = [18, 25, 45, 60, 100] # 100 pour couvrir les âges aberrants
    labels = ['Jeune', 'Adulte', 'Senior', 'Âgé']
    data['Categorie_age'] = pd.cut(data['Age'], bins=bins, labels=labels, right=False)
    
    # Vérification finale de cohérence pour Annees_experience vs Age
    mask_exp_incoherent = data['Annees_experience'] > (data['Age'] - 16)
    data.loc[mask_exp_incoherent, 'Annees_experience'] = (data.loc[mask_exp_incoherent, 'Age'] - 16 - np.random.randint(0,2, size=mask_exp_incoherent.sum())).clip(lower=0)
    
    # S'assurer que les inactifs ont bien un revenu bas ou NaN
    inactifs_mask = data['CSP'] == 'Inactifs'
    if 'Revenu_Annuel' in data.columns:
        data.loc[inactifs_mask & data['Revenu_Annuel'].notna(), 'Revenu_Annuel'] = \
            data.loc[inactifs_mask & data['Revenu_Annuel'].notna(), 'Revenu_Annuel'].apply(lambda x: min(x, 12000))

    # AJOUT: Colonnes Redondantes et Non Pertinentes
    data['Revenu_Mensuel'] = (data['Revenu_Annuel'] / 12).round(2)
    data['Adresse_Email'] = [f"user{uuid.uuid4().hex[:6]}@example.com" for _ in range(N_RECORDS)]
    data['CIN'] = [f"{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}{random.randint(100000,999999)}" for _ in range(N_RECORDS)]

    ordered_columns = [
        'Age', 'Categorie_age', 'Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial',
        'Niveau_education', 'Annees_experience', 'CSP', 'Secteur_emploi',
        'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole',
        'Revenu_secondaire', 'Revenu_Annuel', 'Revenu_Mensuel',  # Ajout de Revenu_Mensuel
        'Adresse_Email', 'CIN'  # Ajout de Adresse_Email et CIN
    ]
    final_columns = [col for col in ordered_columns if col in data.columns]
    missing_cols = [col for col in data.columns if col not in final_columns] # Au cas où
    data = data[final_columns + missing_cols]

    return data

# --- Main Execution ---
if __name__ == "__main__":
    print(f"Génération de {N_RECORDS} enregistrements avec le script ...")
    dataset = generate_dataset()
    
    print("\n--- Statistiques Descriptives du Revenu Annuel  ---")
    print(dataset['Revenu_Annuel'].describe())
    
    print("\nMoyennes par Milieu :")
    print(dataset.groupby('Milieu')['Revenu_Annuel'].mean())

    print("\nMoyennes par CSP :")
    print(dataset.groupby('CSP')['Revenu_Annuel'].mean().sort_values(ascending=False))

    print("\nMoyennes par Niveau d'éducation :")
    print(dataset.groupby('Niveau_education')['Revenu_Annuel'].mean().sort_values(ascending=False))

    dataset.to_csv(FILENAME, index=False, encoding='utf-8')
    print(f"\nDataset '{FILENAME}' généré avec succès ({len(dataset)} enregistrements).")


Génération de 40000 enregistrements avec le script ...

--- Statistiques Descriptives du Revenu Annuel  ---
count     40000.000000
mean      23234.973550
std       31689.439974
min         200.000000
25%        5155.000000
50%        9419.500000
75%       30203.000000
max      700000.000000
Name: Revenu_Annuel, dtype: float64

Moyennes par Milieu :
Milieu
Rural     22149.102970
Urbain    23963.650457
Name: Revenu_Annuel, dtype: float64

Moyennes par CSP :
CSP
Cadres supérieurs             110439.000631
Professions intermédiaires     41111.379008
Employés                       18994.951390
Ouvriers                        6637.720935
Inactifs                        4499.000000
Agriculteurs                    4181.118777
Name: Revenu_Annuel, dtype: float64

Moyennes par Niveau d'éducation :
Niveau_education
Supérieur      68052.903471
Secondaire     21305.731931
Fondamental     5933.697770
Sans niveau     3753.893059
Name: Revenu_Annuel, dtype: float64

Dataset 'dataset_revenu_marocains.c

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

def load_and_preprocess_data(file_path="dataset_revenu_marocains_.csv"):
    """
    Charge et prétraite le dataset pour la modélisation.
    """
    # 1. Charger le dataset
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{file_path}' n'a pas été trouvé. Veuillez d'abord exécuter generate_dataset_.py.")
        return None, None, None, None
    
    print("Dataset chargé avec succès.")

    # 2. Séparer X et y, et supprimer 'Categorie_age'
    if 'Revenu_Annuel' not in df.columns:
        print("Erreur : La colonne 'Revenu_Annuel' (cible) est manquante dans le dataset.")
        return None, None, None, None
        
    y = df['Revenu_Annuel'].copy()
    X = df.drop(columns=['Revenu_Annuel', 'Categorie_age'], errors='ignore')
    print("Variable cible et caractéristiques séparées.")

    # 3. Diviser en ensembles d'entraînement et de test (avant toute imputation/scaling)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Données divisées : X_train shape {X_train.shape}, X_test shape {X_test.shape}")

    # Faire des copies pour éviter SettingWithCopyWarning lors des modifications
    X_train = X_train.copy()
    X_test = X_test.copy()

    # 4. Identifier les types de colonnes à partir de X_train
    numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Colonnes numériques identifiées : {numerical_cols}")
    print(f"Colonnes catégorielles identifiées : {categorical_cols}")

    # 5. Imputation des valeurs manquantes
    # Imputation numérique
    if numerical_cols:
        num_imputer = SimpleImputer(strategy='median')
        X_train[numerical_cols] = num_imputer.fit_transform(X_train[numerical_cols])
        X_test[numerical_cols] = num_imputer.transform(X_test[numerical_cols])
        print("Imputation des valeurs manquantes numériques terminée (médiane).")

    # Imputation catégorielle
    if categorical_cols:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
        X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])
        print("Imputation des valeurs manquantes catégorielles terminée (mode).")

    # 6. Encodage One-Hot des variables catégorielles
    if categorical_cols:
        X_train = pd.get_dummies(X_train, columns=categorical_cols, prefix=categorical_cols, dummy_na=False)
        X_test = pd.get_dummies(X_test, columns=categorical_cols, prefix=categorical_cols, dummy_na=False)
        print("Encodage One-Hot terminé.")

        # 7. Alignement des colonnes après one-hot encoding
        # S'assure que X_test a les mêmes colonnes que X_train, dans le même ordre,
        # en ajoutant les colonnes manquantes avec 0 et en supprimant celles en trop.
        train_cols_after_dummies = X_train.columns.tolist()
        X_test = X_test.reindex(columns=train_cols_after_dummies, fill_value=0)
        # S'assurer que X_train n'a pas de colonnes qui ne seraient pas dans X_test après réindexation
        # (normalement géré par la ligne ci-dessus, mais pour être sûr)
        X_train = X_train[train_cols_after_dummies] 
        print("Colonnes alignées entre les ensembles d'entraînement et de test.")
        print(f"Nouvelles dimensions après encodage : X_train shape {X_train.shape}, X_test shape {X_test.shape}")


    # 8. Standardisation des variables numériques (celles d'origine)
    # Ne pas standardiser les colonnes créées par get_dummies (qui sont 0/1)
    if numerical_cols: # Utilise la liste originale des colonnes numériques
        scaler = StandardScaler()
        X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
        X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
        print("Standardisation des colonnes numériques d'origine terminée.")
        
    # Gestion des NaN dans y (normalement, y ne devrait pas avoir de NaN si la génération est correcte)
    # Si y_train ou y_test ont des NaN, cela peut poser problème aux modèles.
    # Une stratégie simple est de les supprimer (avec les lignes X correspondantes) AVANT le split.
    # Ici, nous supposons que y est propre. Si ce n'est pas le cas, il faudrait ajouter une étape de nettoyage de y.
    if y_train.isnull().any() or y_test.isnull().any():
        print("Attention : Des valeurs NaN sont présentes dans la variable cible y_train ou y_test. Cela peut nécessiter un traitement supplémentaire.")


    print("Prétraitement terminé.")
    return X_train, X_test, y_train, y_test

if __name__ == "__main__":
    print("Exécution du script de prétraitement...")
    X_train, X_test, y_train, y_test = load_and_preprocess_data()

    if X_train is not None:
        print("\n--- Informations sur les données prétraitées ---")
        print("X_train shape:", X_train.shape)
        print("X_test shape:", X_test.shape)
        print("y_train shape:", y_train.shape)
        print("y_test shape:", y_test.shape)
        
        print("\nPremières lignes de X_train (après prétraitement):")
        print(X_train.head())
        
        print("\nStatistiques de y_train:")
        print(y_train.describe())

        # Optionnel : Sauvegarder les fichiers prétraités
        # X_train.to_csv("X_train_processed.csv", index=False)
        # X_test.to_csv("X_test_processed.csv", index=False)
        # y_train.to_csv("y_train.csv", index=False, header=True)
        # y_test.to_csv("y_test.csv", index=False, header=True)
        # print("\nFichiers prétraités sauvegardés (décommenter pour activer).")

Exécution du script de prétraitement...
Dataset chargé avec succès.
Variable cible et caractéristiques séparées.
Données divisées : X_train shape (32000, 13), X_test shape (8000, 13)
Colonnes numériques identifiées : ['Age', 'Annees_experience']
Colonnes catégorielles identifiées : ['Sexe', 'Milieu', 'Region_geographique', 'Etat_matrimonial', 'Niveau_education', 'CSP', 'Secteur_emploi', 'Propriete_immobiliere', 'Vehicule_motorise', 'Terrain_agricole', 'Revenu_secondaire']
Imputation des valeurs manquantes numériques terminée (médiane).
Imputation des valeurs manquantes catégorielles terminée (mode).
Encodage One-Hot terminé.
Colonnes alignées entre les ensembles d'entraînement et de test.
Nouvelles dimensions après encodage : X_train shape (32000, 36), X_test shape (8000, 36)
Standardisation des colonnes numériques d'origine terminée.
Prétraitement terminé.

--- Informations sur les données prétraitées ---
X_train shape: (32000, 36)
X_test shape: (8000, 36)
y_train shape: (32000,)
y_te

In [38]:
# CELLULE 3 : Importations supplémentaires et initialisation pour la modélisation

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time

# Pour stocker les performances des modèles
model_performance = {}

# Définir une constante pour la reproductibilité si nécessaire dans les modèles
RANDOM_STATE = 42

# Vérifier si les données sont chargées (elles devraient l'être par la cellule précédente)
if 'X_train' not in locals() or \
   'X_test' not in locals() or \
   'y_train' not in locals() or \
   'y_test' not in locals():
    print("ERREUR : Les données X_train, X_test, y_train, y_test ne sont pas chargées.")
    print("Veuillez exécuter la cellule de prétraitement des données (load_and_preprocess_data).")
else:
    print("Données d'entraînement et de test prêtes pour la modélisation.")
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Données d'entraînement et de test prêtes pour la modélisation.
X_train shape: (32000, 36), y_train shape: (32000,)
X_test shape: (8000, 36), y_test shape: (8000,)


In [39]:
# CELLULE 4 : 3.1. Régression Linéaire (Modèle de Base)

if 'X_train' in locals():
    print("\n--- 3.1. Entraînement du Modèle de Régression Linéaire ---")
    
    # Initialiser et entraîner le modèle
    linear_model = LinearRegression()
    
    start_time_lr = time.time()
    linear_model.fit(X_train, y_train)
    end_time_lr = time.time()
    
    training_time_lr = end_time_lr - start_time_lr
    print(f"Entraînement terminé en {training_time_lr:.2f} secondes.")
    
    # Prédictions
    predictions_lr = linear_model.predict(X_test)
    
    # Évaluation
    mae_lr = mean_absolute_error(y_test, predictions_lr)
    rmse_lr = np.sqrt(mean_squared_error(y_test, predictions_lr))
    r2_lr = r2_score(y_test, predictions_lr)
    
    print("\n--- Évaluation du Modèle de Régression Linéaire ---")
    print(f"MAE (Mean Absolute Error): {mae_lr:.2f} DH")
    print(f"RMSE (Root Mean Squared Error): {rmse_lr:.2f} DH")
    print(f"R² (Coefficient de détermination): {r2_lr:.4f}")
    
    # Stocker les performances
    model_performance['Linear Regression'] = {
        'MAE': mae_lr,
        'RMSE': rmse_lr,
        'R2': r2_lr,
        'Training Time (s)': training_time_lr,
        'Best Params': 'N/A'
    }
    
    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Régression Linéaire) ---")
    df_predictions_lr = pd.DataFrame({'Réel': y_test.values, 'Prédit_LR': predictions_lr})
    display(df_predictions_lr.head())
else:
    print("Veuillez d'abord charger et prétraiter les données.")



--- 3.1. Entraînement du Modèle de Régression Linéaire ---
Entraînement terminé en 0.16 secondes.

--- Évaluation du Modèle de Régression Linéaire ---
MAE (Mean Absolute Error): 4758.18 DH
RMSE (Root Mean Squared Error): 10938.45 DH
R² (Coefficient de détermination): 0.8806

--- Aperçu des Prédictions vs Valeurs Réelles (Régression Linéaire) ---


,Réel,Prédit_LR
0,5191.0,3300.258708
1,6581.0,5060.956705
2,44592.0,41689.349111
3,7755.0,7252.506818
4,5916.0,6197.278566


In [40]:
# CELLULE 5 : 3.2. Arbres de Décision (DecisionTreeRegressor) avec Ajustement des Hyperparamètres

if 'X_train' in locals():
    print("\n--- 3.2. Entraînement et Ajustement des Hyperparamètres pour DecisionTreeRegressor ---")

    # Définir la grille d'hyperparamètres selon les spécifications du projet
    param_grid_dt = {
        'criterion': ['squared_error'], # 'friedman_mse' est aussi une option
        'max_depth': [5],
        'min_samples_split': [2]
    }
    # Note: 'absolute_error' pour criterion peut être significativement plus lent.

    # Initialiser GridSearchCV
    grid_search_dt = GridSearchCV(
        estimator=DecisionTreeRegressor(random_state=RANDOM_STATE),
        param_grid=param_grid_dt,
        cv=5, # Nombre de plis pour la validation croisée
        scoring='neg_mean_squared_error', # Métrique pour l'évaluation
        n_jobs=-1, # Utiliser tous les processeurs disponibles
        verbose=1 # Afficher des messages pendant l'ajustement
    )

    print(f"Ajustement des hyperparamètres pour DecisionTreeRegressor en cours (CV={grid_search_dt.cv})...")
    start_time_dt_grid = time.time()
    grid_search_dt.fit(X_train, y_train)
    end_time_dt_grid = time.time()
    grid_search_time_dt = end_time_dt_grid - start_time_dt_grid
    print(f"Ajustement terminé en {grid_search_time_dt:.2f} secondes.")

    # Meilleurs hyperparamètres et score
    print(f"\nMeilleurs hyperparamètres trouvés pour DecisionTreeRegressor : {grid_search_dt.best_params_}")
    # Le best_score_ est neg_mean_squared_error, donc plus proche de 0 est mieux.
    print(f"Meilleur score (neg_mean_squared_error) de validation croisée : {grid_search_dt.best_score_:.4f}")
    
    # Meilleur estimateur
    best_dt_model = grid_search_dt.best_estimator_
    
    # Prédictions sur l'ensemble de test avec le meilleur modèle
    predictions_dt = best_dt_model.predict(X_test)
    
    # Évaluation du meilleur modèle
    mae_dt = mean_absolute_error(y_test, predictions_dt)
    rmse_dt = np.sqrt(mean_squared_error(y_test, predictions_dt))
    r2_dt = r2_score(y_test, predictions_dt)
    
    print("\n--- Évaluation du Modèle DecisionTreeRegressor (avec meilleurs hyperparamètres) ---")
    print(f"MAE: {mae_dt:.2f} DH")
    print(f"RMSE: {rmse_dt:.2f} DH")
    print(f"R²: {r2_dt:.4f}")
    
    # Stocker les performances
    model_performance['Decision Tree (Tuned)'] = {
        'MAE': mae_dt,
        'RMSE': rmse_dt,
        'R2': r2_dt,
        'Training Time (s)': grid_search_time_dt, # Temps pour GridSearchCV
        'Best Params': grid_search_dt.best_params_
    }

    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Decision Tree) ---")
    df_predictions_dt = pd.DataFrame({'Réel': y_test.values, 'Prédit_DT': predictions_dt})
    display(df_predictions_dt.head(20))
else:
    print("Veuillez d'abord charger et prétraiter les données.")



--- 3.2. Entraînement et Ajustement des Hyperparamètres pour DecisionTreeRegressor ---
Ajustement des hyperparamètres pour DecisionTreeRegressor en cours (CV=5)...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0

Ajustement terminé en 5.24 secondes.

Meilleurs hyperparamètres trouvés pour DecisionTreeRegressor : {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_split': 2}
Meilleur score (neg_mean_squared_error) de validation croisée : -34773682.8128

--- Évaluation du Modèle DecisionTreeRegressor (avec meilleurs hyperparamètres) ---
MAE: 2732.17 DH
RMSE: 7851.75 DH
R²: 0.9385

--- Aperçu des Prédictions vs Valeurs Réelles (Decision Tree) ---


,Réel,Prédit_DT
0,5191.0,5850.070002
1,6581.0,5850.070002
2,44592.0,48508.989062
3,7755.0,7893.825496
4,5916.0,5850.070002
5,19169.0,17418.653620
6,36117.0,29110.664239
7,24724.0,17418.653620
8,29488.0,28249.900657
9,123182.0,113072.856410


In [41]:
# CELLULE 7 : 3.3. Forêts Aléatoires (RandomForestRegressor) avec Ajustement des Hyperparamètres

if 'X_train' in locals():
    print("\n--- 3.3. Entraînement et Ajustement des Hyperparamètres pour RandomForestRegressor ---")
    from sklearn.ensemble import RandomForestRegressor

    # Définir une grille d'hyperparamètres plus ciblée pour RandomForest
    # RandomForest peut être long à entraîner avec une grille large.
    param_grid_rf = {
        'n_estimators': [50, 100, 150, 200], # Nombre d'arbres dans la forêt
        'criterion': ['squared_error'], # 'absolute_error' est très lent pour RF, 'friedman_mse' est une alternative
        'max_depth': [None, 5, 10, 15, 20],      # Profondeur max des arbres (similaire à DT)
        # 'min_samples_split': [2, 4], # Nombre min d'échantillons pour diviser un nœud
        # 'min_samples_leaf': [1, 2], # Nombre min d'échantillons par feuille (peut être ajouté)
        # 'max_features': ['sqrt', 'log2'] # Nombre de features à considérer pour la meilleure division (peut être ajouté)
    }
    # Pour un premier essai, cette grille est raisonnable.
    # Si le temps le permet, vous pouvez l'étendre.

    # Initialiser GridSearchCV
    grid_search_rf = GridSearchCV(
        estimator=RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1), # n_jobs dans RF pour l'entraînement des arbres
        param_grid=param_grid_rf,
        cv=3, # Réduire le nombre de plis pour accélérer si nécessaire (5 est standard)
        scoring='neg_mean_squared_error',
        n_jobs=-1, # n_jobs dans GridSearchCV pour les plis de CV
        verbose=1
    )

    print(f"Ajustement des hyperparamètres pour RandomForestRegressor en cours (CV={grid_search_rf.cv})...")
    start_time_rf_grid = time.time()
    grid_search_rf.fit(X_train, y_train)
    end_time_rf_grid = time.time()
    grid_search_time_rf = end_time_rf_grid - start_time_rf_grid
    print(f"Ajustement terminé en {grid_search_time_rf:.2f} secondes.")

    # Meilleurs hyperparamètres et score
    print(f"\nMeilleurs hyperparamètres trouvés pour RandomForestRegressor : {grid_search_rf.best_params_}")
    print(f"Meilleur score (neg_mean_squared_error) de validation croisée : {grid_search_rf.best_score_:.4f}")
    
    # Meilleur estimateur
    best_rf_model = grid_search_rf.best_estimator_
    
    # Prédictions
    predictions_rf = best_rf_model.predict(X_test)
    
    # Évaluation
    mae_rf = mean_absolute_error(y_test, predictions_rf)
    rmse_rf = np.sqrt(mean_squared_error(y_test, predictions_rf))
    r2_rf = r2_score(y_test, predictions_rf)
    
    print("\n--- Évaluation du Modèle RandomForestRegressor (avec meilleurs hyperparamètres) ---")
    print(f"MAE: {mae_rf:.2f} DH")
    print(f"RMSE: {rmse_rf:.2f} DH")
    print(f"R²: {r2_rf:.4f}")
    
    # Stocker les performances
    model_performance['Random Forest (Tuned)'] = {
        'MAE': mae_rf,
        'RMSE': rmse_rf,
        'R2': r2_rf,
        'Training Time (s)': grid_search_time_rf,
        'Best Params': grid_search_rf.best_params_
    }

    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Random Forest) ---")
    df_predictions_rf = pd.DataFrame({'Réel': y_test.values, 'Prédit_RF': predictions_rf})
    display(df_predictions_rf.head(20))

    # Mettre à jour et afficher le tableau comparatif
    print("\n--- Tableau Comparatif des Performances des Modèles (mis à jour) ---")
    performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
    display(performance_df.sort_values(by='RMSE', ascending=True))

else:
    print("Veuillez d'abord charger et prétraiter les données.")



--- 3.3. Entraînement et Ajustement des Hyperparamètres pour RandomForestRegressor ---
Ajustement des hyperparamètres pour RandomForestRegressor en cours (CV=3)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Ajustement terminé en 165.90 secondes.

Meilleurs hyperparamètres trouvés pour RandomForestRegressor : {'criterion': 'squared_error', 'max_depth': 10, 'n_estimators': 200}
Meilleur score (neg_mean_squared_error) de validation croisée : -26128557.4194

--- Évaluation du Modèle RandomForestRegressor (avec meilleurs hyperparamètres) ---
MAE: 1036.76 DH
RMSE: 7202.94 DH
R²: 0.9482

--- Aperçu des Prédictions vs Valeurs Réelles (Random Forest) ---


,Réel,Prédit_RF
0,5191.0,4627.772926
1,6581.0,6733.117870
2,44592.0,45384.846785
3,7755.0,7663.381414
4,5916.0,5155.897502
5,19169.0,17345.140658
6,36117.0,35403.239039
7,24724.0,23714.834923
8,29488.0,29058.456263
9,123182.0,124688.332603



--- Tableau Comparatif des Performances des Modèles (mis à jour) ---


,MAE,RMSE,R2,Training Time (s),Best Params
Random Forest (Tuned),1036.762160,7202.940191,0.948209,165.899335,"{'criterion': 'squared_error', 'max_depth': 10..."
Decision Tree (Tuned),2732.168530,7851.749472,0.938458,5.241215,"{'criterion': 'squared_error', 'max_depth': 5,..."
Linear Regression,4758.180573,10938.450755,0.880560,0.159706,N/A


In [45]:
# CELLULE 8 : 3.4. Gradient Boosting Regressor avec Ajustement des Hyperparamètres

if 'X_train' in locals():
    print("\n--- 3.4. Entraînement et Ajustement des Hyperparamètres pour GradientBoostingRegressor ---")
    from sklearn.ensemble import GradientBoostingRegressor

    # Définir une grille d'hyperparamètres pour GradientBoostingRegressor
    # Gradient Boosting peut aussi être long à entraîner.
    param_grid_gbr = {
        'loss': ['squared_error'], # Ajout de 'absolute_error'
        'n_estimators': [100, 200, 300],          
        'learning_rate': [0.01, 0.1, 0.2],                
        'subsample': [0.5, 0.8, 1.0]            
    }
    # Cette grille est un bon point de départ.

    # Initialiser GridSearchCV
    grid_search_gbr = GridSearchCV(
        estimator=GradientBoostingRegressor(random_state=RANDOM_STATE),
        param_grid=param_grid_gbr,
        cv=3, # Maintenir cv=3 pour la vitesse, augmenter à 5 si le temps le permet
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )

    print(f"Ajustement des hyperparamètres pour GradientBoostingRegressor en cours (CV={grid_search_gbr.cv})...")
    start_time_gbr_grid = time.time()
    grid_search_gbr.fit(X_train, y_train)
    end_time_gbr_grid = time.time()
    grid_search_time_gbr = end_time_gbr_grid - start_time_gbr_grid
    print(f"Ajustement terminé en {grid_search_time_gbr:.2f} secondes.")

    # Meilleurs hyperparamètres et score
    print(f"\nMeilleurs hyperparamètres trouvés pour GradientBoostingRegressor : {grid_search_gbr.best_params_}")
    print(f"Meilleur score (neg_mean_squared_error) de validation croisée : {grid_search_gbr.best_score_:.4f}")
    
    # Meilleur estimateur
    best_gbr_model = grid_search_gbr.best_estimator_
    
    # Prédictions
    predictions_gbr = best_gbr_model.predict(X_test)
    
    # Évaluation
    mae_gbr = mean_absolute_error(y_test, predictions_gbr)
    rmse_gbr = np.sqrt(mean_squared_error(y_test, predictions_gbr))
    r2_gbr = r2_score(y_test, predictions_gbr)
    
    print("\n--- Évaluation du Modèle GradientBoostingRegressor (avec meilleurs hyperparamètres) ---")
    print(f"MAE: {mae_gbr:.2f} DH")
    print(f"RMSE: {rmse_gbr:.2f} DH")
    print(f"R²: {r2_gbr:.4f}")
    
    # Stocker les performances
    model_performance['Gradient Boosting (Tuned)'] = {
        'MAE': mae_gbr,
        'RMSE': rmse_gbr,
        'R2': r2_gbr,
        'Training Time (s)': grid_search_time_gbr,
        'Best Params': grid_search_gbr.best_params_
    }

    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (Gradient Boosting) ---")
    df_predictions_gbr = pd.DataFrame({'Réel': y_test.values, 'Prédit_GBR': predictions_gbr})
    display(df_predictions_gbr.head(20))

    # Mettre à jour et afficher le tableau comparatif
    print("\n--- Tableau Comparatif des Performances des Modèles (mis à jour) ---")
    performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
    display(performance_df.sort_values(by='RMSE', ascending=True))

else:
    print("Veuillez d'abord charger et prétraiter les données.")



--- 3.4. Entraînement et Ajustement des Hyperparamètres pour GradientBoostingRegressor ---
Ajustement des hyperparamètres pour GradientBoostingRegressor en cours (CV=3)...
Fitting 3 folds for each of 27 candidates, totalling 81 fits


0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.01s - Debugger warning: It seems that frozen modules are being used, which may
0

Ajustement terminé en 125.82 secondes.

Meilleurs hyperparamètres trouvés pour GradientBoostingRegressor : {'learning_rate': 0.1, 'loss': 'squared_error', 'n_estimators': 300, 'subsample': 1.0}
Meilleur score (neg_mean_squared_error) de validation croisée : -21200511.8167

--- Évaluation du Modèle GradientBoostingRegressor (avec meilleurs hyperparamètres) ---
MAE: 825.42 DH
RMSE: 6789.42 DH
R²: 0.9540

--- Aperçu des Prédictions vs Valeurs Réelles (Gradient Boosting) ---


,Réel,Prédit_GBR
0,5191.0,5134.509174
1,6581.0,6925.558081
2,44592.0,45360.253308
3,7755.0,8113.605842
4,5916.0,5937.097866
5,19169.0,18849.403859
6,36117.0,37045.816293
7,24724.0,24121.518125
8,29488.0,31133.773082
9,123182.0,123694.611260



--- Tableau Comparatif des Performances des Modèles (mis à jour) ---


,MAE,RMSE,R2,Training Time (s),Best Params
Gradient Boosting (Tuned),825.420814,6789.417881,0.953985,125.820957,"{'learning_rate': 0.1, 'loss': 'squared_error'..."
MLP Regressor (Tuned),927.481741,6898.479328,0.952494,144.379020,"{'activation': 'relu', 'alpha': 0.001, 'hidden..."
Random Forest (Tuned),1036.762160,7202.940191,0.948209,165.899335,"{'criterion': 'squared_error', 'max_depth': 10..."
Decision Tree (Tuned),2732.168530,7851.749472,0.938458,5.241215,"{'criterion': 'squared_error', 'max_depth': 5,..."
Linear Regression,4758.180573,10938.450755,0.880560,0.159706,N/A


In [ ]:
# CELLULE 9 : 3.5. Réseaux de Neurones (MLPRegressor) avec Ajustement des Hyperparamètres

if 'X_train' in locals():
    print("\n--- 3.5. Entraînement et Ajustement des Hyperparamètres pour MLPRegressor ---")
    from sklearn.neural_network import MLPRegressor
    from sklearn.preprocessing import StandardScaler # S'assurer que les données sont bien standardisées pour MLP

    # Note: Les données X_train, X_test devraient déjà être standardisées par preprocess_data_.py
    # Si ce n'était pas le cas, il faudrait le faire ici spécifiquement pour MLP.

    # Définir une grille d'hyperparamètres TRÈS SIMPLE pour MLPRegressor pour un premier essai
    # L'exploration d'architectures de réseau peut être très longue.
    param_grid_mlp = { # Remplacer par la grille simplifiée
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate_init': [0.001, 0.01],
        'max_iter': [300, 500], # S'assurer que early_stopping est activé dans l'estimateur
        # 'batch_size': [32, 64] # Pourrait être testé dans une seconde passe
    }

    grid_search_mlp = GridSearchCV(
        estimator=MLPRegressor(random_state=RANDOM_STATE, early_stopping=True, n_iter_no_change=15, tol=1e-4), # Ajuster n_iter_no_change et tol
        param_grid=param_grid_mlp,
        cv=2, # Maintenir cv=2 ou 3 pour la vitesse
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )

    print(f"Ajustement des hyperparamètres pour MLPRegressor en cours (CV={grid_search_mlp.cv})...")
    print("Cela peut prendre beaucoup de temps...")
    start_time_mlp_grid = time.time()
    grid_search_mlp.fit(X_train, y_train) # Utiliser X_train_scaled
    end_time_mlp_grid = time.time()
    grid_search_time_mlp = end_time_mlp_grid - start_time_mlp_grid
    print(f"Ajustement terminé en {grid_search_time_mlp:.2f} secondes.")

    # Meilleurs hyperparamètres et score
    print(f"\nMeilleurs hyperparamètres trouvés pour MLPRegressor : {grid_search_mlp.best_params_}")
    print(f"Meilleur score (neg_mean_squared_error) de validation croisée : {grid_search_mlp.best_score_:.4f}")
    
    # Meilleur estimateur
    best_mlp_model = grid_search_mlp.best_estimator_
    
    # Prédictions
    predictions_mlp = best_mlp_model.predict(X_test) # Utiliser X_test_scaled
    
    # Évaluation
    mae_mlp = mean_absolute_error(y_test, predictions_mlp)
    rmse_mlp = np.sqrt(mean_squared_error(y_test, predictions_mlp))
    r2_mlp = r2_score(y_test, predictions_mlp)
    
    print("\n--- Évaluation du Modèle MLPRegressor (avec meilleurs hyperparamètres) ---")
    print(f"MAE: {mae_mlp:.2f} DH")
    print(f"RMSE: {rmse_mlp:.2f} DH")
    print(f"R²: {r2_mlp:.4f}")
    
    # Stocker les performances
    model_performance['MLP Regressor (Tuned)'] = {
        'MAE': mae_mlp,
        'RMSE': rmse_mlp,
        'R2': r2_mlp,
        'Training Time (s)': grid_search_time_mlp,
        'Best Params': grid_search_mlp.best_params_
    }

    print("\n--- Aperçu des Prédictions vs Valeurs Réelles (MLP Regressor) ---")
    df_predictions_mlp = pd.DataFrame({'Réel': y_test.values, 'Prédit_MLP': predictions_mlp})
    display(df_predictions_mlp.head(20))

    # Mettre à jour et afficher le tableau comparatif
    print("\n--- Tableau Comparatif des Performances des Modèles (mis à jour) ---")
    performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
    display(performance_df.sort_values(by='RMSE', ascending=True))

else:
    print("Veuillez d'abord charger et prétraiter les données.")



--- 3.5. Entraînement et Ajustement des Hyperparamètres pour MLPRegressor ---
Ajustement des hyperparamètres pour MLPRegressor en cours (CV=2)...
Cela peut prendre beaucoup de temps...
Fitting 2 folds for each of 48 candidates, totalling 96 fits


0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
0.02s - Debugger warning: It seems that frozen modules are being used, which may
0

KeyboardInterrupt: 

In [42]:
# CELLULE 6 : Affichage du Tableau Comparatif des Performances

if model_performance:
    print("\n--- Tableau Comparatif des Performances des Modèles ---")
    performance_df = pd.DataFrame.from_dict(model_performance, orient='index')
    display(performance_df.sort_values(by='RMSE', ascending=True))
else:
    print("Aucun modèle n'a encore été entraîné et évalué.")



--- Tableau Comparatif des Performances des Modèles ---


,MAE,RMSE,R2,Training Time (s),Best Params
Random Forest (Tuned),1036.762160,7202.940191,0.948209,165.899335,"{'criterion': 'squared_error', 'max_depth': 10..."
Decision Tree (Tuned),2732.168530,7851.749472,0.938458,5.241215,"{'criterion': 'squared_error', 'max_depth': 5,..."
Linear Regression,4758.180573,10938.450755,0.880560,0.159706,N/A
